### What hyperparameters to tune:
Take a look here:
> https://stats.stackexchange.com/questions/349761/reasonable-hyperparameter-range-for-latent-dirichlet-allocation

So our go to are:
- Topics number
- alpha: Document-Topic Density
- beta: Word-Topic Density


### Parameters definition

In [1]:
from main.hp_tuning import UniqueParametersConfigFsGenerator, RandomTunableOffsetParameter

seed = 1408
config_generator = UniqueParametersConfigFsGenerator(100, "./output/config", "seen_configurations.noun_only.json")

# The amount of topics we want to look for
# Topics are by far the most relevant parameter to tune, alpha and beta will come later maybe:
config_generator.add_parameter('topics', RandomTunableOffsetParameter(value_range=(7, 72), step=5, seed=seed))
# config_generator.add_parameter('alpha', RandomTunableOffsetParameter(value_range=(0.005, 1.0), step=0.5, seed=seed))
# config_generator.add_parameter('beta', RandomTunableDiscreteParameter(values_list=beta, seed=seed))

In [ ]:
from hp_tuning import LDATuningProcedure
import pandas as pd

corpus = pd.read_csv("../dataset/output/pos_tagged/pre_processed.310k.noun_only.csv")
procedure = LDATuningProcedure(generator=config_generator, top=[3, 10, 25], folds=5)
results = procedure.run(corpus, 10)  # Try 10 different configurations.
procedure.store_results("./output/config/hp_tuning_results.noun_only.json")

In [ ]:
# Show results in graph...
results

In [ ]:
from main.hp_tuning import UniqueParametersConfigFsGenerator, RandomTunableOffsetParameter

seed = 1408
config_generator = UniqueParametersConfigFsGenerator(
    100, "./output/config", "seen_configurations.noun_only_sentence.json"
)

# The amount of topics we want to look for
# Topics are by far the most relevant parameter to tune, alpha and beta will come later maybe:
config_generator.add_parameter('topics', RandomTunableOffsetParameter(value_range=(7, 72), step=5, seed=seed))
# config_generator.add_parameter('alpha', RandomTunableOffsetParameter(value_range=(0.005, 1.0), step=0.5, seed=seed))
# config_generator.add_parameter('beta', RandomTunableDiscreteParameter(values_list=beta, seed=seed))

In [ ]:
from hp_tuning import LDATuningProcedure
import pandas as pd

corpus = pd.read_csv("../dataset/output/pos_tagged_sentence_level/pre_processed.310k.noun_only.csv")
procedure = LDATuningProcedure(generator=config_generator, top=[3, 10, 25], folds=5)
results = procedure.run(corpus, 10)  # Try 10 different configurations.
procedure.store_results("./output/config/hp_tuning_results.noun_only_sent.json")

In [ ]:
# Show results in graph...
results

In [ ]:
from main.hp_tuning import UniqueParametersConfigFsGenerator, RandomTunableOffsetParameter

seed = 1408
config_generator = UniqueParametersConfigFsGenerator(
    100, "./output/config", "seen_configurations.default_sentence.json"
)

# The amount of topics we want to look for
# Topics are by far the most relevant parameter to tune, alpha and beta will come later maybe:
config_generator.add_parameter('topics', RandomTunableOffsetParameter(value_range=(7, 72), step=5, seed=seed))
# config_generator.add_parameter('alpha', RandomTunableOffsetParameter(value_range=(0.005, 1.0), step=0.5, seed=seed))
# config_generator.add_parameter('beta', RandomTunableDiscreteParameter(values_list=beta, seed=seed))

In [ ]:
from hp_tuning import LDATuningProcedure
import pandas as pd

corpus = pd.read_csv("../dataset/output/default_sentences/pre_processed.310k.csv")
procedure = LDATuningProcedure(generator=config_generator, top=[3, 10, 25], folds=5)
results = procedure.run(corpus, 10)  # Try 10 different configurations.
procedure.store_results("./output/config/hp_tuning_results.default_sentence.json")

In [ ]:
# Show results in graph...
results

In [ ]:
#todo fix
# https://www.kaggle.com/code/vijaylokithrr/topic-modelling#7)-Hyperparameter-Tuning
plt.figure(figsize=(20, 7))
ax = sns.lineplot(x=num_topics, y=hyper_para_coherence, label="topic coherences")
ax.axes.set_title('Coherence per Number of Topics', fontsize=25)
ax.set_ylabel('Coherence', fontsize=20)
ax.set_xlabel('Number of Topics', fontsize=20)
plt.show()